# Naive Bayes Classification

## Which of these subject lines are from spam emails?

- Need just a little help? Independent senior living might be the answer
- Hard sql from today's interview
- Beautiful Women, Discrete Service
- Ever consider driving with Lyft? Apply here.
- 3/2/18 All Hands Unanswered Questions
- Sizwe Documents


What tipped you off?

Say we have a collection of labeled text documents, each one belonging to a category (for example, news articles & which section of the paper they're in). Now we get a new, unlabeled article. How do use what we've seen so far to predict which category it belongs to?

- Do a bunch of complicated part of speech tagging & account for the sequence of words?
- Make the dumbest "bag of words" assumption possible and hope it works?
  - yep this is what we do

Naive Bayes is an extremely simple amazingly effective machine learning technique.

## When do we use it?
1. n << p
2. n small
3. n large
4. streams of input data (online learning)
5. multi-class
6. low memory applications

## What is it really?

It is just [Maximum A Posteriori estimation](https://en.wikipedia.org/wiki/Maximum_a_posteriori_estimation), with a fun approximation to make it easy to calculate.

### Notation: a document $\vec{x}$ is a list of words $\{w_1, w_2, \ldots, w_k\}$, and we want the probability that it belongs to class $y$

#### Examples: 
- e-mail: spam or not spam
- news article: is it from the World sections or Sport or Arts, etc
- essay from anonymous author: is the author really a famous person (whose corpus I have access to)?

## $$P(y|\vec{x}) = \frac{P(\vec{x}|y)P(y)}{P(\vec{x})}$$

Now here's the wacky part: let's assume the features (words) are totally independent of each other. Then we can write

## $$P(\vec{x}|y) = P(w_1|y)\times P(w_2|y)\times\cdots\times P(w_k|y) $$

This is a pretty brazenly naive assumption. It is assuming, for example, that the probability of seeing the word "ball" in an article about sports $P(ball|\text{sports})$ is totally independent of the presence of the word "soccer" in the article.

But if it works, it works.

To classify a document $\vec{x}$, we just need to calculate the posterior probabilities for each class $\{y_j\}$, then see which class has the highest posterior probability.

# $$P(y_j|\vec{x}) \propto P(w_1|y_j)\times P(w_2|y_j)\times\cdots\times P(w_k|y_j)\times P(y_j)  $$

Note that we've dropped the denominator $P(\vec{x})$ since it doesn't depend on $y_j$, and we're only interested in the $y_j$ that maximizes $P(y_j|\vec{x})$.

# What's the prior, $P(y_j)$ ?

The prior probability of class $y$ is simply how frequent that class is among our training documents:
$$P(y_j) = \frac{\text{# of documents of class}\, y_j}{\text{total # of documents}} $$

# What about these likelihoods? How do we calculate 'em?

## Option 1: Multinomial Naive Bayes

Imagine that a document $\vec{x}$ of class $y_j$ is generated by drawing words from a bag (with replacement) with probabilities $P(w_i | y_j)$

Let our vocabulary (the set of unique words observed across the entire training corpus) be $p$ words long. Let's write $\vec{x}$ as a p-dimensional vector of word counts: $\vec{x} = [x_1, x_2, \ldots, x_p]$

Then we can write our posterior (which, remember, is the prior times the likelihood) as
### $$P(y_j|\vec{x}) \propto P(y_j)\times\prod_{i=1}^p P(w_i|y_j)^{x_i} $$

$$\text{log}(P(y_j|\vec{x})) \propto \text{log}(P(y_j)) + \sum_{i=1}^p x_i \text{log}(P(w_i|y_j))$$

To estimate $P(w_i|y_j)$, we could just use 
$$\frac{N_{ji}}{N_j} = \frac{\text{total count of w_i across all documents of class}\, y_j}{\text{total count of all words across all documents of class}\,y_j} $$

But what if our new document contains a word we've never seen before? Then our estimate of $P(w|y)$ for that word would be zero, and then the entire product $P(y|\vec{x})$ would be zero. 

To avoid this, we employ *Laplace Smoothing*
### $$ P(w_i | y_j) = \frac{\alpha + N_{ji}}{\alpha p + N_j} $$

Usually, $\alpha = 1$

## Option 2: Bernoulli Naive Bayes


Imagine, now, that a document of class $y_j$ has a probability $P(w_i|y_j)$ of containing word $w_i$ at least once (regardless of word count).

Again let our vocabulary (the set of unique words observed across the entire training corpus) be $p$ words long. Let's write $\vec{x}$ as a p-dimensional vector of word *occurrences*: $x_i = 1$ if the $w_i$ is in that document at all, $x_i = 0$ if not: $\vec{x} = [x_1, x_2, \ldots, x_p]$

Then we can think of our document as a series of Bernoulli trials, one for each word, and our posterior is
### $$P(y_j|\vec{x}) \propto P(y_j) \times\prod_{i=1}^p P(w_i|y_j)^{x_i}(1 - P(w_i|y_j))^{1 - x_i} $$

$$\text{log}(P(y_j|\vec{x})) \propto \text{log}(P(y_j)) + \sum_{i=1}^p x_i \text{log}(P(w_i|y_j)) + (1 - x_i)\text{log}(1 - P(w_i|y_j))$$

To estimate $P(w_i|y_j)$, we could just use 
$$\frac{D_{ji}}{D_j} = \frac{\text{total # of documents of class}\,y_j\text{ containing}\, w_i}{\text{total number of documents of class}\,y_j} $$

But again we'd hit the "unseen word" problem. So our smoothing here looks like
### $$ P(w_i | y_j) = \frac{\alpha + D_{ji}}{2\alpha + D_j} $$

Again with $\alpha = 1$ usually.

## Which is better?
- Bernoulli tends to work better with shorter documents...
- ...but sklearn says "It is advisable to evaluate both models, if time permits."

References:

[sklearn User Guide: Naive Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html)

[Spam Filtering with Naive Bayes – Which Naive Bayes?](http://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=E870DFA148786F5A27F88CF80FB4D73B?doi=10.1.1.61.5542&rep=rep1&type=pdf)

## Other flavors: Gaussian Naive Bayes
When my features are continuous (but my target is still a class), I can go ahead and assume that data for a given class has normally distributed values for each feature.

You cannot stop me from making this assumption.

Then my likelihood above is just a product of Gaussian probability density functions. Same process!

[See here for bugs](http://www.cs.ucr.edu/~eamonn/CE/Bayesian%20Classification%20withInsect_examples.pdf)

### Let's code!

In [1]:
X = ['when its time to party we will party hard',
          'theres a party in my mind',
           'i need something to change my mind']
y = ['andrew wk', 'talking heads', 'talking heads']

In [2]:
x_test = ['a change in the weather']

In [3]:
from collections import Counter, defaultdict

In [4]:
vocab = set()
for row in X:
    for word in row.split():
        vocab.add(word)

In [5]:
vocab

{'a',
 'change',
 'hard',
 'i',
 'in',
 'its',
 'mind',
 'my',
 'need',
 'party',
 'something',
 'theres',
 'time',
 'to',
 'we',
 'when',
 'will'}

In [6]:
set(' '.join(X).split())

{'a',
 'change',
 'hard',
 'i',
 'in',
 'its',
 'mind',
 'my',
 'need',
 'party',
 'something',
 'theres',
 'time',
 'to',
 'we',
 'when',
 'will'}

In [7]:
class_counts = Counter(y)

In [8]:
class_counts

Counter({'andrew wk': 1, 'talking heads': 2})

In [9]:
total_docs = sum(class_counts.values())

In [10]:
words_per_class = defaultdict(int)
word_count_per_class = defaultdict(Counter)

for row, label in zip(X,y):
    words_per_class[label] += len(row.split())
    word_count_per_class[label].update(Counter(row.split()))

In [11]:
words_per_class

defaultdict(int, {'andrew wk': 9, 'talking heads': 13})

In [12]:
word_count_per_class

defaultdict(collections.Counter,
            {'andrew wk': Counter({'hard': 1,
                      'its': 1,
                      'party': 2,
                      'time': 1,
                      'to': 1,
                      'we': 1,
                      'when': 1,
                      'will': 1}),
             'talking heads': Counter({'a': 1,
                      'change': 1,
                      'i': 1,
                      'in': 1,
                      'mind': 2,
                      'my': 2,
                      'need': 1,
                      'party': 1,
                      'something': 1,
                      'theres': 1,
                      'to': 1})})

In [13]:
x_test_vocab_counts = defaultdict(Counter)
for elm in x_test:
    x_test_word_counts = Counter(elm.split())
    for word in vocab:
        x_test_vocab_counts[elm][word] = x_test_word_counts[word] # x_test_word_counts[elm] = Counter(elm.split())

x_test_vocab_counts

defaultdict(collections.Counter,
            {'a change in the weather': Counter({'a': 1,
                      'change': 1,
                      'hard': 0,
                      'i': 0,
                      'in': 1,
                      'its': 0,
                      'mind': 0,
                      'my': 0,
                      'need': 0,
                      'party': 0,
                      'something': 0,
                      'theres': 0,
                      'time': 0,
                      'to': 0,
                      'we': 0,
                      'when': 0,
                      'will': 0})})

#### Making a prediction using Multinomial Naive Bayes

In [21]:
alpha = 1

for x_test in x_test_vocab_counts:
    for y_class in class_counts:
        prior = class_counts[y_class]/total_docs
        posterior = prior
        for word, word_count in x_test_vocab_counts[x_test].items():
            N_ji = word_count_per_class[y_class][word] # total count of word_i across all documents of class y_j
            N_j = words_per_class[y_class] # total count of all words across all documents of class y_j
            posterior *= ((alpha + N_ji)/(alpha*len(vocab) + N_j))**word_count
        print(y_class, posterior)

andrew wk 1.8965255651646187e-05
talking heads 0.00019753086419753083


#### Which do we predict?